In [1]:
using JLD2, DataFrames
keys(load("../data/spy_clean_valid_iv.jld2"))

KeySet for a Dict{String, Any} with 1 entry. Keys:
  "df_clean"

In [34]:
@load "../data/spy_clean_valid_iv.jld2" df_clean

1-element Vector{Symbol}:
 :df_clean

In [19]:
first(df,5)

Row,Column1,level_0,QUOTE_UNIXTIME,QUOTE_READTIME,QUOTE_DATE,QUOTE_TIME_HOURS,UNDERLYING_LAST,EXPIRE_DATE,EXPIRE_UNIX,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,P_BID,P_ASK,P_SIZE,P_LAST,P_DELTA,P_GAMMA,P_VEGA,P_THETA,P_RHO,P_IV,P_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,r
,Int64,Int64,Int64,String31,Date,Float64,Float64,Date,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String15,Float64,Float64,Float64,Float64,Float64,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?
1,0,0,1664827200,2022-10-03 16:00,2022-10-03,16.0,366.62,2022-10-03,1664827200,0.0,0.996,0.0033,0.00208,-0.03475,0.00693,0.22754,1946.0,9.16,6 x 1,9.2,10.12,357.0,0.01,0.01,0 x 4555,0.01,-0.00706,0.00519,0.00353,-0.00989,0.0,0.24586,31198.0,9.6,0.026,3.46
2,1,1,1664827200,2022-10-03 16:00,2022-10-03,16.0,366.62,2022-10-03,1664827200,0.0,0.99543,0.00479,0.00259,-0.03543,0.00745,0.18367,2031.0,7.65,327 x 1,7.22,8.1,359.0,0.01,0.01,0 x 2240,0.01,-0.00917,0.00725,0.00351,-0.00951,-0.0004,0.19937,27324.0,7.6,0.021,3.46
3,2,2,1664827200,2022-10-03 16:00,2022-10-03,16.0,366.62,2022-10-03,1664827200,0.0,0.99052,0.00887,0.0041,-0.03935,0.00668,0.17477,8527.0,6.5,399 x 10,6.28,7.05,360.0,0.01,0.01,0 x 3756,0.01,-0.0101,0.00915,0.00476,-0.00978,-0.00016,0.17624,74485.0,6.6,0.018,3.46
4,6,6,1664827200,2022-10-03 16:00,2022-10-03,16.0,366.62,2022-10-03,1664827200,0.0,0.02337,0.04667,0.00869,-0.01041,-0.00026,0.07128,53740.0,0.01,0 x 1241,0.01,0.01,369.0,2.52,2.74,6 x 2,2.45,-0.81014,0.10289,0.04424,-0.25543,-0.00488,0.16372,3973.0,2.4,0.006,3.46
5,7,7,1664827200,2022-10-03 16:00,2022-10-03,16.0,366.62,2022-10-03,1664827200,0.0,0.01828,0.02773,0.00698,-0.01041,-0.00021,0.09707,42283.0,0.01,0 x 1567,0.01,0.01,370.0,3.52,3.69,10 x 10,3.89,-0.84859,0.07293,0.03858,-0.23049,-0.00446,0.1974,2655.0,3.4,0.009,3.46


In [20]:
for col in names(df)
    println(col)
end

Column1
level_0
QUOTE_UNIXTIME
QUOTE_READTIME
QUOTE_DATE
QUOTE_TIME_HOURS
UNDERLYING_LAST
EXPIRE_DATE
EXPIRE_UNIX
DTE
C_DELTA
C_GAMMA
C_VEGA
C_THETA
C_RHO
C_IV
C_VOLUME
C_LAST
C_SIZE
C_BID
C_ASK
STRIKE
P_BID
P_ASK
P_SIZE
P_LAST
P_DELTA
P_GAMMA
P_VEGA
P_THETA
P_RHO
P_IV
P_VOLUME
STRIKE_DISTANCE
STRIKE_DISTANCE_PCT
r


In [30]:
first(df[:,[:UNDERLYING_LAST,:STRIKE,:DTE,:r,:C_IV,:P_IV]],90)

Row,UNDERLYING_LAST,STRIKE,DTE,r,C_IV,P_IV
,Float64,Float64,Float64,Float64?,Float64,Float64
1,366.62,357.0,0.0,3.46,0.22754,0.24586
2,366.62,359.0,0.0,3.46,0.18367,0.19937
3,366.62,360.0,0.0,3.46,0.17477,0.17624
4,366.62,369.0,0.0,3.46,0.07128,0.16372
5,366.62,370.0,0.0,3.46,0.09707,0.1974
6,366.62,372.0,0.0,3.46,0.14341,0.32225
7,366.62,373.0,0.0,3.46,0.18994,0.27726
8,366.62,374.0,0.0,3.46,0.20768,0.48653
9,366.62,375.0,0.0,3.46,0.22253,0.34203


In [22]:
using JLD2, DataFrames, Statistics  # ← Add Statistics here

println("🔍 CHECKING FOR INVALID IV VALUES\n")

# 1. Find negative IVs
negative_call_iv = filter(row -> row.C_IV < 0, df)
negative_put_iv = filter(row -> row.P_IV < 0, df)

println("Negative Call IVs: $(nrow(negative_call_iv))")
println("Negative Put IVs: $(nrow(negative_put_iv))")

if nrow(negative_call_iv) > 0
    println("\n📋 Sample negative Call IV rows:")
    println(first(select(negative_call_iv, :QUOTE_DATE, :STRIKE, :C_IV, :C_BID, :C_ASK, :C_VOLUME), 5))
end

# 2. Check for zeros
zero_call_iv = filter(row -> row.C_IV == 0, df)
println("\nZero Call IVs: $(nrow(zero_call_iv))")

# 3. Check for extremely low values
very_low_iv = filter(row -> 0 < row.C_IV < 0.01, df)  # < 1%
println("Very low Call IVs (<1%): $(nrow(very_low_iv))")

# 4. Check distribution
println("\n📊 IV Distribution:")
println("  Min:    $(minimum(df.C_IV))")
println("  1st %:  $(quantile(df.C_IV, 0.01))")
println("  5th %:  $(quantile(df.C_IV, 0.05))")
println("  Median: $(median(df.C_IV))")
println("  Mean:   $(round(mean(df.C_IV), digits=3))")
println("  95th %: $(quantile(df.C_IV, 0.95))")
println("  Max:    $(maximum(df.C_IV))")

# 5. Look at characteristics of bad IV rows
if nrow(negative_call_iv) > 0
    println("\n🔍 Characteristics of negative IV rows:")
    println("  Avg volume: $(round(mean(negative_call_iv.C_VOLUME), digits=1))")
    println("  Avg bid: \$$(round(mean(negative_call_iv.C_BID), digits=2))")
    println("  Avg strike distance: $(round(mean(abs.(negative_call_iv.STRIKE_DISTANCE_PCT)), digits=1))%")
end

🔍 CHECKING FOR INVALID IV VALUES

Negative Call IVs: 0
Negative Put IVs: 0

Zero Call IVs: 0
Very low Call IVs (<1%): 0

📊 IV Distribution:
  Min:    0.05005
  1st %:  0.08923
  5th %:  0.11629
  Median: 0.22649
  Mean:   0.277
  95th %: 0.62965
  Max:    1.5


In [23]:
println("="^60)
println("DATA CHECK: Do I have everything for Black-Scholes?")
println("="^60 * "\n")

# Check one sample row
row = df[1000, :]  # Random row

println("📊 Sample Option Contract:\n")
println("Date: $(row.QUOTE_DATE)")
println("Expiry: $(row.EXPIRE_DATE) ($(row.DTE) days)")
println()

println("Black-Scholes Inputs:")
println("  S (spot):        \$$(round(row.UNDERLYING_LAST, digits=2))")
println("  K (strike):      \$$(round(row.STRIKE, digits=2))")
println("  T (time):        $(round(row.DTE/365, digits=3)) years")
println("  r (rate):        $(round(row.r*100, digits=2))%")
println("  q (dividend):    1.8% (approximate)")
println("  σ (impl vol):    $(round(row.C_IV, digits=3)) ✅ (already have!)")
println()

println("Market Prices:")
println("  Call Bid:        \$$(round(row.C_BID, digits=2))")
println("  Call Ask:        \$$(round(row.C_ASK, digits=2))")
println("  Call Mid:        \$$(round((row.C_BID + row.C_ASK)/2, digits=2))")
println()

println("Greeks (for validation later):")
println("  Delta:           $(round(row.C_DELTA, digits=3))")
println("  Gamma:           $(round(row.C_GAMMA, digits=4))")
println("  Vega:            $(round(row.C_VEGA, digits=2))")
println()

println("="^60)
println("✅ YOU HAVE EVERYTHING NEEDED FOR BLACK-SCHOLES!")
println("="^60)
println()

println("IV Statistics:")
println("  Min IV:  $(round(minimum(df.C_IV), digits=3))")
println("  Mean IV: $(round(mean(df.C_IV), digits=3))")
println("  Max IV:  $(round(maximum(df.C_IV), digits=3))")
println()

println("🎯 NEXT STEP: Implement black_scholes_call() function")
println("   Use σ = row.C_IV (the implied volatility)")

DATA CHECK: Do I have everything for Black-Scholes?

📊 Sample Option Contract:

Date: 2022-10-03
Expiry: 2022-11-04 (32.0 days)

Black-Scholes Inputs:
  S (spot):        $366.62
  K (strike):      $364.0
  T (time):        0.088 years
  r (rate):        346.0%
  q (dividend):    1.8% (approximate)
  σ (impl vol):    0.267 ✅ (already have!)

Market Prices:
  Call Bid:        $13.58
  Call Ask:        $13.92
  Call Mid:        $13.75

Greeks (for validation later):
  Delta:           0.571
  Gamma:           0.0134
  Vega:            0.43

✅ YOU HAVE EVERYTHING NEEDED FOR BLACK-SCHOLES!

IV Statistics:
  Min IV:  0.05
  Mean IV: 0.277
  Max IV:  1.5

🎯 NEXT STEP: Implement black_scholes_call() function
   Use σ = row.C_IV (the implied volatility)


In [32]:
@save "../data/spy_clean_valid_iv.jld2" df_clean=df


In [ ]:
println("="^60)
println("FINAL DATA CLEANING FOR BLACK-SCHOLES")
println("="^60 * "\n")

initial = nrow(df_clean)
println("Starting rows: $initial\n")

# 1. Add computed columns
println("1️⃣ Adding computed columns...")
df_clean.C_MID = (df_clean.C_BID .+ df_clean.C_ASK) ./ 2
df_clean.P_MID = (df_clean.P_BID .+ df_clean.P_ASK) ./ 2
df_clean.T = df_clean.DTE ./ 365.0  # Time in years
println("   ✅ Added C_MID, P_MID, T\n")

# 2. Filter DTE range (your project: 30-60 days)
println("2️⃣ Filtering DTE to 30-60 days...")
df_filtered = filter(row -> 30 <= row.DTE <= 60, df_clean)
println("   Removed: $(initial - nrow(df_filtered)) rows")
println("   Remaining: $(nrow(df_filtered)) ($(round(100*nrow(df_filtered)/initial, digits=1))%)\n")

# 3. Filter moneyness (near ATM only)
println("3️⃣ Filtering to near-ATM (±15%)...")
prev = nrow(df_filtered)
df_filtered = filter(row -> abs(row.STRIKE_DISTANCE_PCT) <= 15, df_filtered)
println("   Removed: $(prev - nrow(df_filtered)) rows")
println("   Remaining: $(nrow(df_filtered))\n")

# 4. Summary statistics
println("="^60)
println("FINAL DATASET SUMMARY")
println("="^60)
println("\n📊 Size:")
println("   Rows: $(nrow(df_filtered))")
println("   Reduction: $(round(100*(1 - nrow(df_filtered)/initial), digits=1))% of original")

println("\n📅 Date Coverage:")
println("   Range: $(extrema(df_filtered.QUOTE_DATE))")
println("   Trading days: $(length(unique(df_filtered.QUOTE_DATE)))")

println("\n⏱️  DTE:")
println("   Range: $(extrema(df_filtered.DTE))")
println("   Mean: $(round(mean(df_filtered.DTE), digits=1)) days")

println("\n📈 IV:")
println("   Range: $(round(minimum(df_filtered.C_IV), digits=3)) - $(round(maximum(df_filtered.C_IV), digits=3))")
println("   Mean: $(round(mean(df_filtered.C_IV), digits=3))")

println("\n💰 Price:")
println("   Call mid range: \$$(round(minimum(df_filtered.C_MID), digits=2)) - \$$(round(maximum(df_filtered.C_MID), digits=2))")
println("   Call mid mean: \$$(round(mean(df_filtered.C_MID), digits=2))")

println("\n🎯 Moneyness:")
println("   Strike distance: $(round(minimum(df_filtered.STRIKE_DISTANCE_PCT), digits=1))% to $(round(maximum(df_filtered.STRIKE_DISTANCE_PCT), digits=1))%")

println("\n📝 Sample row:")
sample = first(df_filtered, 1)
println("   Date: $(sample.QUOTE_DATE[1])")
println("   S: \$$(round(sample.UNDERLYING_LAST[1], digits=2))")
println("   K: \$$(round(sample.STRIKE[1], digits=2))")
println("   DTE: $(sample.DTE[1]) days (T = $(round(sample.T[1], digits=3)) years)")
println("   IV: $(round(sample.C_IV[1], digits=3))")
println("   r: $(round(sample.r[1]*100, digits=2))%")
println("   Market price: \$$(round(sample.C_MID[1], digits=2))")

# 5. Save
println("\n💾 Saving final dataset...")
@save "../data/spy_ready_for_bs.jld2" df
println("   ✅ Saved: data/processed/spy_ready_for_bs.jld2")

println("\n" * "="^60)
println("✅ DATA READY FOR BLACK-SCHOLES IMPLEMENTATION!")
println("="^60 * "\n")



In [2]:
@load "../data/spy_ready_for_bs.jld2" df

1-element Vector{Symbol}:
 :df

In [ ]:
function black_scholes_call(S, K, T, r, q, σ)
    # Step 1: Calculate d1
    d1 = (log(S/K) + (r - q + σ^2/2) * T) / (σ * sqrt(T))
 
    
    # Step 2: Calculate d2
    d2 = d1 - σ*sqrt(T)
    
    # Step 3: Calculate N(d1) and N(d2)
    # These are cumulative normal distribution values
    # Hint: You need to import Distributions.jl
    # N(x) = cdf(Normal(), x)
    
    # Step 4: Calculate call price using formula
    # price = S*exp(-q*T)*N(d1) - K*exp(-r*T)*N(d2)
    
    # Step 5: Return the price
    return price
end